In [1]:
pip install mediapipe

  Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached jax-0.6.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.6.1-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.2-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl (51.0 MB)
Using cached sounddevice-0.5.2-py3-none-win_amd64.whl (363 kB)
Using cached absl_py-2.3.0-py3-none-any.whl (135 kB)
Using cached jax-0.6.1-py3-none-any.whl (2.4 MB)
Using cached jaxlib-0.6.1-cp312-cp312-win_amd64.whl (56.9 MB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl (46.2 MB)
Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl (210 kB)
Note: you may need to restart the kernel to use updated 

In [2]:
pip install opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

colors = [
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (0, 255, 255)
]
color_index = 0
canvas = None
x1, y1 = 0, 0

def draw_buttons(frame):
    cv2.rectangle(frame, (10, 1), (110, 65), colors[0], -1)
    cv2.putText(frame, "BLUE", (20, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.rectangle(frame, (120, 1), (220, 65), colors[1], -1)
    cv2.putText(frame, "GREEN", (130, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.rectangle(frame, (230, 1), (330, 65), colors[2], -1)
    cv2.putText(frame, "RED", (250, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.rectangle(frame, (340, 1), (440, 65), colors[3], -1)
    cv2.putText(frame, "YELLOW", (345, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
    cv2.rectangle(frame, (450, 1), (550, 65), (0, 0, 0), -1)
    cv2.putText(frame, "CLEAR", (460, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    if canvas is None:
        canvas = np.zeros_like(frame)

    draw_buttons(frame)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            h, w, _ = frame.shape
            cx, cy = int(index_finger.x * w), int(index_finger.y * h)
            cv2.circle(frame, (cx, cy), 8, colors[color_index], -1)

            if cy <= 65:
                if 10 < cx < 110:
                    color_index = 0
                elif 120 < cx < 220:
                    color_index = 1
                elif 230 < cx < 330:
                    color_index = 2
                elif 340 < cx < 440:
                    color_index = 3
                elif 450 < cx < 550:
                    canvas = np.zeros_like(frame)
                x1, y1 = 0, 0
            else:
                if x1 == 0 and y1 == 0:
                    x1, y1 = cx, cy
                else:
                    canvas = cv2.line(canvas, (x1, y1), (cx, cy), colors[color_index], 5)
                    x1, y1 = cx, cy
    else:
        x1, y1 = 0, 0

    frame = cv2.add(frame, canvas)
    cv2.imshow("Air Canvas with Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 